In [67]:
from pymongo import MongoClient

In [78]:
# Connection URL
connection_url = "mongodb://localhost:27017/"
client = MongoClient(connection_url)

In [79]:
db_names=client.list_database_names()
db_names

['admin', 'config', 'local', 'tmdb_db']

In [80]:
db = client['tmdb_db']
collection = db['movies']

Notar que nuestra colección de películas guarda 3 tipos de objeto:
* Película
* Género
* Actor

In [81]:
collection.distinct('type')

['actor', 'genre']

In [82]:
# Aggregation pipeline to get top 5 unique popular movies
pipeline = [
    {"$group": {
        "_id": "$title",
        "document": {"$first": "$$ROOT"},  # Retain the first document encountered
        "maxPopularity": {"$max": "$popularity"}  # Get max popularity for the title
    }},
    {"$sort": {"maxPopularity": -1}},  # Sort by popularity descending
    {"$limit": 10},  # Limit to top 5
    {"$replaceRoot": {"newRoot": "$document"}}  # Replace the root to output the whole document
]

# Execute the aggregation pipeline
top_movies = collection.aggregate(pipeline)
print("\nTop 10 most popular movies:")
for movie in top_movies:
    print(f"  {movie['title']} ({movie['release_date']}), Popularity: {movie['popularity']}")


Top 10 most popular movies:
  Kingdom of the Planet of the Apes (2024-05-08), Popularity: 3629.389
  Godzilla x Kong: The New Empire (2024-03-27), Popularity: 3148.883
  Civil War (2024-04-10), Popularity: 2767.933
  The Fall Guy (2024-04-24), Popularity: 1915.521
  Furiosa: A Mad Max Saga (2024-05-22), Popularity: 1193.177
  Godzilla Minus One (2023-11-03), Popularity: 755.261
  Dune: Part Two (2024-02-27), Popularity: 874.167
  Un père idéal (2024-04-21), Popularity: 905.771
  Noryang: Deadly Sea (2023-12-20), Popularity: 692.948
  Kung Fu Panda 4 (2024-03-02), Popularity: 752.116


In [83]:
pipeline = [
    {"$match": {"type": {"$exists": False}}},  # Assuming 'type' does not exist in movie documents
    
    {"$lookup": {
        "from": "movies",  # Assuming genres are in the same collection
        "localField": "genre_ids",
        "foreignField": "id",
        "as": "genre_details"
    }},
    
    {"$unwind": "$genre_details"},

    {"$match": {"genre_details.type": "genre"}},  # Ensure only genre documents are used

    {"$unwind": "$genre_ids"},
    
    {"$match": {"$expr": {"$eq": ["$genre_ids", "$genre_details.id"]}}},

    {"$group": {
        "_id": "$genre_details.name",
        "AverageRating": {"$avg": "$vote_average"}
    }},
    
    {"$sort": {"AverageRating": -1}}
]

average_ratings_by_genre = collection.aggregate(pipeline)
print("Average Ratings by Genre:\n")
for genre in average_ratings_by_genre:
    print(f"Genre: {genre['_id']} - Average Rating: {genre['AverageRating']:.2f}")



Average Ratings by Genre:

Genre: History - Average Rating: 7.84
Genre: Documentary - Average Rating: 7.41
Genre: Crime - Average Rating: 7.37
Genre: War - Average Rating: 7.24
Genre: Drama - Average Rating: 7.23
Genre: Science Fiction - Average Rating: 7.14
Genre: Adventure - Average Rating: 7.07
Genre: Mystery - Average Rating: 7.06
Genre: Comedy - Average Rating: 7.02
Genre: Action - Average Rating: 6.91
Genre: Romance - Average Rating: 6.67
Genre: Thriller - Average Rating: 6.58
Genre: Fantasy - Average Rating: 6.33
Genre: Family - Average Rating: 6.15
Genre: Horror - Average Rating: 6.05
Genre: Animation - Average Rating: 6.04
Genre: TV Movie - Average Rating: 5.78


In [84]:
pipeline = [
    # Match movies with a high rating
    {"$match": {
        "vote_average": {"$gte": 7.5},  # Movies with a vote average of 7.5 or higher
        "type": {"$exists": False}  # Ensure we're looking at movie documents
    }},
    
    # Lookup genres
    {"$lookup": {
        "from": "movies",  # Assuming all data is in the same collection
        "localField": "genre_ids",
        "foreignField": "id",
        "as": "genre_details"
    }},
    
    # Lookup actors
    {"$lookup": {
        "from": "movies",
        "localField": "actor_ids",
        "foreignField": "id",
        "as": "actor_details"
    }},
    
    # Project the required fields
    {"$project": {
        "title": 1,
        "vote_average": 1,
        "genres": "$genre_details.name",  # Extracting only the genre names
        "actors": "$actor_details.name"  # Extracting only the actor names
    }},
    
    # Optional: limit the results to top 10 movies
    {"$limit": 10}
]

# Execute the aggregation pipeline
top_movies = collection.aggregate(pipeline)

# Print the results
print("Highly Rated Movies with Genre and Actor Details:")
for movie in top_movies:
    print(movie)


Highly Rated Movies with Genre and Actor Details:
{'_id': ObjectId('665691c79efa5d94c5b45f6e'), 'title': 'Furiosa: A Mad Max Saga', 'vote_average': 7.7, 'genres': ['Science Fiction', 'Action', 'Adventure', 'Science Fiction', 'Adventure', 'Action', 'Action', 'Action', 'Action', 'Adventure', 'Science Fiction', 'Science Fiction', 'Action', 'Action', 'Science Fiction', 'Adventure', 'Action', 'Action', 'Adventure', 'Action', 'Action', 'Science Fiction', 'Action', 'Action', 'Action', 'Adventure', 'Action', 'Science Fiction', 'Science Fiction', 'Adventure', 'Action', 'Science Fiction', 'Action', 'Action', 'Adventure', 'Science Fiction', 'Action', 'Science Fiction', 'Action', 'Science Fiction', 'Action', 'Action', 'Adventure', 'Action', 'Adventure', 'Adventure', 'Action', 'Action', 'Adventure', 'Science Fiction', 'Action', 'Adventure', 'Science Fiction', 'Action', 'Adventure', 'Action', 'Science Fiction', 'Science Fiction', 'Action', 'Action', 'Adventure', 'Adventure', 'Action', 'Science Ficti

In [74]:
b=collection.find({'type': 'actor'})
for i in b:
    print(i)

{'_id': ObjectId('66569ed3c9b3bb4f1b4d2634'), 'type': 'actor', 'id': 15556, 'name': 'Rebecca Hall'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d2635'), 'type': 'actor', 'id': 226366, 'name': 'Brian Tyree Henry'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d2636'), 'type': 'actor', 'id': 221018, 'name': 'Dan Stevens'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d2637'), 'type': 'actor', 'id': 2948491, 'name': 'Kaylee Hottle'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d2638'), 'type': 'actor', 'id': 60416, 'name': 'Alex Ferns'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d2639'), 'type': 'actor', 'id': 123701, 'name': 'Fala Chen'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d263a'), 'type': 'actor', 'id': 15298, 'name': 'Rachel House'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d263b'), 'type': 'actor', 'id': 2896645, 'name': 'Ron Smyck'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d263c'), 'type': 'actor', 'id': 1984017, 'name': 'Chantelle Jamieson'}
{'_id': ObjectId('66569ed3c9b3bb4f1b4d263d'), 'type': 'actor', 'id': 2096500, 'name': 'G

In [75]:
# Look for any potential references to actors
sample_movies = collection.find({}).limit(5)

print("Sample Movies:")
for movie in sample_movies:
    print(movie)

# Check a sample actor document
sample_actor = db['movies'].find_one({"type": "actor"})
print("\nSample Actor Document:")
print(sample_actor)

Sample Movies:
{'_id': ObjectId('665691c79efa5d94c5b45f6a'), 'adult': False, 'backdrop_path': '/xRd1eJIDe7JHO5u4gtEYwGn5wtf.jpg', 'genre_ids': [878, 28, 12], 'id': 823464, 'original_language': 'en', 'original_title': 'Godzilla x Kong: The New Empire', 'overview': 'Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', 'popularity': 3148.883, 'poster_path': '/z1p34vh7dEOnLDmyCrlUVLuoDzd.jpg', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'video': False, 'vote_average': 7.265, 'vote_count': 2243, 'actors': [{'id': 15556, 'name': 'Rebecca Hall'}, {'id': 226366, 'name': 'Brian Tyree Henry'}, {'id': 221018, 'name': 'Dan Stevens'}, {'id': 2948491, 'name': 'Kaylee Hottle'}, {'id': 60416, 'name': 'Alex Ferns'}, {'id': 123701, 'name': 'Fala Chen'}, {'id': 15298, 'name': 'Rachel House'}, {'id': 2896645, 'name': 'Ron Smyck'}, {'id': 1984017, 'na

In [87]:
pipeline = [
    # Filter movies based on a specific criterion (e.g., titles containing "Kong")
    {"$match": {"title": {"$regex": "Dune", "$options": "i"}}},  # Example filter

    # Lookup to join with genres
    {"$lookup": {
        "from": "movies",  # adjust if genres are in a separate collection
        "localField": "genre_ids",
        "foreignField": "id",
        "as": "genre_details"
    }},
    
    # Lookup to join with actors
    {"$lookup": {
        "from": "movies",  # adjust if actors are in a separate collection
        "localField": "actor_ids",
        "foreignField": "id",
        "as": "actor_details"
    }},
    
    # Project to flatten arrays and include only necessary fields
    {"$project": {
        "title": 1,
        "release_date": 1,
        "genres": "$genre_details.name",
        "actors": "$actor_details.name",
        "overview": 1
    }},
    
    # Unwind genres and actors to group and filter unique values
    {"$unwind": "$genres"},
    {"$unwind": "$actors"},
    
    # Group by movie details and collect unique genres and actors
    {"$group": {
        "_id": {
            "title": "$title",
            "release_date": "$release_date",
            "overview": "$overview"
        },
        "unique_genres": {"$addToSet": "$genres"},
        "unique_actors": {"$addToSet": "$actors"}
    }},
    
    # Project the final desired structure
    {"$project": {
        "_id": 0,
        "title": "$_id.title",
        "release_date": "$_id.release_date",
        "overview": "$_id.overview",
        "genres": "$unique_genres",
        "actors": "$unique_actors"
    }}
]

# Execute the aggregation pipeline
movie_details = db.movies.aggregate(pipeline)

# Print the results
for movie in movie_details:
    print(f"Title: {movie['title']}")
    print(f"Release Date: {movie['release_date']}")
    print(f"Overview: {movie['overview']}")
    print("Genres: " + ", ".join(movie['genres']))
    print("Actors: " + ", ".join(movie['actors']))
    print("-" * 60)


Title: Dune: Part Two
Release Date: 2024-02-27
Overview: Follow the mythic journey of Paul Atreides as he unites with Chani and the Fremen while on a path of revenge against the conspirators who destroyed his family. Facing a choice between the love of his life and the fate of the known universe, Paul endeavors to prevent a terrible future only he can foresee.
Genres: Science Fiction, Adventure
Actors: Zendaya, Souheila Yacoub, Kait Tenison, Amra Mallassi, Omar A.K., Jonathan Gunning, Léa Seydoux, Tara Breathnach, Remi Fadare, Josh Brolin, Huw Novelli, Sára Bácsfalvi, Joelle, Alexandra Tóth, Alison Halstead, Molly Mcowan, Zsófia Kocsis, Dora Kápolnai-Schvab, Oxa Hazel, Vic Zander, Affif Ben Badra, Dominic McHale, Jasper Ryan-Carter, Zouhair Elakkari, Christopher Walken, Tim Hilborne, Kincsö Pethö, Gábor Szemán, Alan Mehdizadeh, Amer El-Erwadi, Hopi Grace, Hamza Sayd, Anton Saunders, Ana Cilas, Imola Gáspár, Tracy Coogan, Rand Faris, Elbooz Omar Ahmed Fathie, Abdellah Echahbi, Giusi Mer

In [ ]:
pipeline = [
    {"$unwind": "$genres"},
    {"$group": {
        "_id": "$genres.name",
        "average_budget": {"$avg": "$budget"}
    }},
    {"$sort": {"average_budget": -1}}
]
average_budget_by_genre = db.movies.aggregate(pipeline)
for genre in average_budget_by_genre:
    print(f"Genre: {genre['_id']}, Average Budget: {genre['average_budget']}")


In [ ]:
pipeline = [
    {"$match": {"type": {"$exists": False}}},
    {"$addFields": {
        "release_year": {
            "$year": {
                "$dateFromString": {
                    "dateString": "$release_date"
                }
            }
        }
    }},
    {"$group": {
        "_id": "$release_year",
        "average_popularity": {"$avg": "$popularity"}
    }},
    {"$sort": {"_id": 1}} 
]

average_popularity_by_year = collection.aggregate(pipeline)

print("Average Popularity by Year:\n")
for year_data in average_popularity_by_year:
    print(f"Year: {year_data['_id']} - Average Popularity: {year_data['average_popularity']:.2f}")


In [ ]:
pipeline = [
    {"$unwind": "$production_companies"},
    {"$group": {
        "_id": "$production_companies.name",
        "average_revenue": {"$avg": "$revenue"}
    }},
    {"$sort": {"average_revenue": -1}},
    {"$limit": 10},
    {"$project": {
        "production_company": "$_id",
        "average_revenue": 1
    }}
]

top_10_companies_by_revenue = db.movies.aggregate(pipeline)

for company in top_10_companies_by_revenue:
    print(f"Production Company: {company['production_company']}, Average Revenue: {company['average_revenue']}")


In [ ]:
pipeline = [
    {"$match": {"vote_average": {"$exists": True}}},
    {"$sort": {"vote_average": -1}},
    {"$limit": 100},
    {"$unwind": "$genres"},
    {"$group": {
        "_id": "$genres.name",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]
common_genres_top_rated = db.movies.aggregate(pipeline)
for genre in common_genres_top_rated:
    print(f"Genre: {genre['_id']}, Count: {genre['count']}")


In [76]:
# Close the connection to MongoDB
client.close()